# Acceleration Classification

### 1. Extract Features and Labels
Relevant Features for Acceleration:
- accelerometerXAxis
- accelerometerYAxis
- accelerometerZAxis
- speedKmh
<br></br>
- timestamp ?
- gyroscope ?

In [112]:
import os
import json
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

data_bmw = []
data_honda = []
labels_bmw = []
labels_honda = []

prepared_data = []
prepared_labels = []

scaler = MinMaxScaler(feature_range=(0, 1))

In [113]:
# Get the current directory path
current_dir = os.getcwd()

# Go up two directories from the current directory
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))

# Now join the desired directory ('Datasets') to the path that is two levels up
datasets_dir = os.path.join(root_dir, 'Datasets')

# Iterate over all files in the directory
for root, dirs, files in os.walk(datasets_dir):
    for file in files:
        if file.endswith('.json'):
            car = os.path.basename(root).split()[0].upper()            
            label = os.path.basename(os.path.dirname(root))
            
            file_data = json.load(open(os.path.join(root, file)))
            file_data = file_data['capturedData']
                                    
            # Convert to dataframe
            file_data = pd.DataFrame(file_data)           
            
            # Drop unnecessary columns
            file_data = file_data.drop(['id', 'Latitude', 'Longitude', 'gyroscopeXAxis', 'gyroscopeYAxis', 'gyroscopeZAxis'], axis=1)
            
            # Rename speed Km/h to speed
            file_data.rename(columns={'speed Km/h': 'speed'}, inplace=True)
            file_data.rename(columns={'speedKmh': 'speed'}, inplace=True)
            
            # Drop timestamp
            file_data = file_data.drop(['createdAt'], axis=1, errors='ignore')
            file_data = file_data.drop(['timestamp'], axis=1, errors='ignore')                    
            
            if car == 'BMW':
                data_bmw.append(file_data.copy())
                labels_bmw.append(label)
            elif car == 'HONDA':
                data_honda.append(file_data.copy())
                labels_honda.append(label)            

#check for NaNs
if data_bmw[0].isnull().values.any() or data_honda[0].isnull().values.any():
    print('NaNs in data')
else:
    print('NO NaNs in data')

NO NaNs in data


### 2. Feature Engineering
New Features:
- Speed (Calculated by accelerometerY)
- Accumulated Acceleration
- Distance moved ?

#### 4. Normalize and Scale Features

In [114]:
# Normalize Each car
data_bmw = pd.concat(data_bmw)
data_honda = pd.concat(data_honda)

# Choose columns to normalize
data_bmw = data_bmw[['accelerometerXAxis', 'accelerometerYAxis', 'accelerometerZAxis', 'speed']]
data_honda = data_honda[['accelerometerXAxis', 'accelerometerYAxis', 'accelerometerZAxis', 'speed']]

# Normalize
scaled_data_bmw = scaler.fit_transform(data_bmw)
prepared_data.append(scaled_data_bmw)
print('Scaled Data BMW: ')
print(scaled_data_bmw)

scaled_data_honda = scaler.fit_transform(data_honda)
prepared_data.append(scaled_data_honda)
print('Scaled Data Honda: ')
print(scaled_data_honda)

# Aggregate all data
prepared_data = np.concatenate(prepared_data)
print('Aggregated Data: ')
print(prepared_data)

prepared_labels = labels_bmw + labels_honda

# Sequence 4by4




Scaled Data BMW: 
[[0.42432991 0.59656404 0.72300274 0.        ]
 [0.43609964 0.5425548  0.73695225 0.        ]
 [0.42231019 0.69527125 0.69672298 0.        ]
 ...
 [0.24617761 0.53066339 0.81379612 0.95426202]
 [0.36474162 0.48877002 0.740912   0.95426202]
 [0.47802334 0.4854856  0.63905244 0.95426202]]
Scaled Data Honda: 
[[0.74303088 0.44561212 0.51776463 0.        ]
 [0.80168045 0.49898038 0.52469522 0.        ]
 [0.80317997 0.50692158 0.47638032 0.00923695]
 ...
 [0.76952134 0.38315083 0.60441686 0.93172695]
 [0.79364197 0.42399552 0.59923704 0.93172695]
 [0.73517428 0.40522551 0.44928707 0.93172695]]
Aggregated Data: 
[[0.42432991 0.59656404 0.72300274 0.        ]
 [0.43609964 0.5425548  0.73695225 0.        ]
 [0.42231019 0.69527125 0.69672298 0.        ]
 ...
 [0.76952134 0.38315083 0.60441686 0.93172695]
 [0.79364197 0.42399552 0.59923704 0.93172695]
 [0.73517428 0.40522551 0.44928707 0.93172695]]
